# Data chunking

## Authors & Contributors
### Authors
- Tina Odaka, Ifremer (France), [@tinaok](https://github.com/tinaok)
- Pier Lorenzo Marasco, Ispra (Italy), [@pl-marasco](https://github.com/pl-marasco)

### Contributors
- Anne Fouilloux, University of Oslo (Norway), [@annefou](https://github.com/annefou)



<div class="alert alert-info">
<i class="fa-question-circle fa" style="font-size: 22px;color:#666;"></i> Overview
    <br>
    <br>
    <b>Questions</b>
    <ul>
        <li>Why do chunking matter?</li>
        <li>How can I read datasets by chunks to optimize memory usage?</li>
    </ul>
    <b>Objectives</b>
    <ul>
        <li>Learn about chunking</li>
        <li>Learn about zarr </li>
        <li>Use kerchunk to consolidate chunk metadata and prepare single ensemble datasets for parallel computing</li>
    </ul>
</div>

## Context

When dealing with large data files or collections, it's often impossible to load all the data you want to analyze into a single computer memory at once.  This is a situation where the Pangeo ecosystem can help you a lot. Xarray offers the possibility to work lazily on data __chunks__, which means pieces of an entire dataset.  By taking a data as 'chunk' we can process our data piece by piece on a single computer using local dask cluster, or distributed computing cluster.  How we will process these 'chunks' in a parallel enviroment will be discussed in [dask_introduction](./dask_introduction.ipynb)  The concept of __chunk__  will be explained here.

When we process our data piece by piece, we would like to have our original data or processed result also save in 'chunks'. [Zarr](https://zarr.readthedocs.io/en/stable/) is a major way used in pangeo ecosystem to save our xarray dataset in __chunk__.

[Zarr](https://zarr.readthedocs.io/en/stable/)  is not the only file format which use __chunk__. We will also be using [kerchunk library](https://fsspec.github.io/kerchunk/) in this notebook to build a virtual __chunked__ dataset based on NetCDF, and show how it optimizes the access and analysis of large datasets.

The analysis is very similar to what we have done in previous episodes, however we will use data on a global coverage and not only on a small geographical area (e.g. Lombardia).

### Data

In this episode, we will be using Global Long Term Statistics (1999-2019) products provided by the [Copernicus Global Land Service](https://land.copernicus.eu/global/index.html) and access them through [S3-comptabile storage](https://en.wikipedia.org/wiki/Amazon_S3) ([OpenStack Object Storage "Swift"](https://wiki.openstack.org/wiki/Swift)) with a data catalog we have created and made publicly available.

## Setup

This episode uses the following main Python packages:

- fsspec {cite:ps}`d-fsspec-2018`
- s3fs {cite:ps}`d-s3fs-2016`
- xarray {cite:ps}`d-xarray-hoyer2017` with [`netCDF4`](https://pypi.org/project/h5netcdf/) and [`h5netcdf`](https://pypi.org/project/h5netcdf/) engines
- dask {cite:ps}`d-dask-2016`
- kerchunk {cite:ps}`d-kerchunk-2021`
- geopandas {cite:ps}`d-geopandas-jordahl2020`
- matplotlib {cite:ps}`d-matplotlib-Hunter2007`

Please install these packages if not already available in your Python environment (see [Setup page](https://pangeo-data.github.io/foss4g-2022/before/setup.html)).

### Packages

In this episode, Python packages are imported when we start to use them. However, for best software practices, we recommend you to install and import all the necessary libraries at the top of your Jupyter notebook.

## Global LTS

In the previous episode, we used Long Term tastitics time-series for the region of Lombardy e.g. a very small area. Now we would like to use the original dataset that has a global coverage. Let us first open a single file (for January 1999-2019) to understand how much larger the global dataset is.

In [ ]:
import fsspec
import s3fs
import xarray as xr

In [ ]:
fs = s3fs.S3FileSystem(anon=True,
      client_kwargs={
         'endpoint_url': 'https://object-store.cloud.muni.cz'
      })

In [ ]:
s3path = 's3://foss4g-data/CGLS_LTS_1999_2019/c_gls_NDVI-LTS_1999-2019-1221_GLOBE_VGT-PROBAV_V3.0.1.nc'

As shown in the [Data discovery](./data_discovery.ipynb) chapter, when we have several files to read at once, we need to use Xarray `open_mfdataset` instead of `open_dataset`. It takes a list in input, and not a single element. We can also use `open_mfdataset` with one file as done below.

In [ ]:
%%time
LTS = xr.open_mfdataset([fs.open(s3path)])
LTS

Using `open_mfdataset` automatically switch from Numpy Arrays to Dask Arrays as the data structure used by Xarray. 

<div class="alert alert-info">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Attention</b>
    <br>
    <ul>
        <li>You can try to open the same file with xr.open_dataset(fs.open(s3path))</li> 
        <li> compre the xarray output between  the open_mfdataset and open_dataset,  what do you see as difference?</li>
            </ul>
</div>

## What is __chunk__

If you look carefully to `LTS`, each Data Variable is a `dask.array` with a chunsize of `(15680, 40320)`. So basically accessing one data variable would load arrays of dimensions `(15680, 40320)` into the memory.  You can have the information by clicking the icon indicated in the image below

![Dask.array](../figures/datasize.png)

When you open a netCDF file, by default, the chunks correspond to the entire size of the dataset originated to one file. When you need to analyze a considerable number of large files, the memory may not be sufficient anymore. This is where understanding chunking comes into play.



__chunk__ is putting a dataset into small pieces. 

original data set is one piece, 
![Dask.array](../figures/notchunked.png)
and we put them into pieces.  
![Dask.array](../figures/chunked.png)

We put them into pieces so that we can process our data block by block.  

We can set chunk also with 'chunk' option when opening a file with  xr.open_mfdataset or even with xr.open_dataset

In [ ]:
xr.open_dataset(fs.open(s3path),
                chunk=(15680,40320))

Lets try to 'chunk' our data array LTS.nobs using following command.
We would like to play with small size, so first we selct our data as you've learnd in [xarray_introduction](./xarray_introduction.ipynb)  

In [ ]:
test=LTS.nobs.sel(lat=slice(80.,70.),lon=slice(70.,90))
test

The test value have dimensions `(1121, 2240)`.  We will chunk it in a dimetion of 600x600 by following command. 

In [ ]:
test=test.chunk(600)
test

As you can see in the above graphics we got 8 __chunk__.  These 8 chunks are noted as (0,0) ..(0,3) (1,0),..(1,3) as you can see in the next visualisation.

In [ ]:
test.data.visualize()

` test.data`  is the backend of Xarray's Data Array, [__Dask Array__](https://docs.dask.org/en/stable/array.html). We will introduce that in next section [dask_introduction](./dask_introduction.ipynb).

In [ ]:
test.data

<div class="alert alert-info">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Attention</b>
    <br>
    <br>
    <ul>You can try to apply different way for specifying chunk on test DataArray
        <br>
        <br>
        <li> chunk = 'auto' -> xarray seek ideal size according to the preferred chunk sizes.</li>
        <li> chunk = -1 -> the entire dimesion lenght will be used as a single chunk,
        <li> chunk= 1Mb -> xarray seek the size according to a specific memory limit expressed in Mb</li>
        <li> chunk={'lat'=-1, 'lon'= 1000} </li>
        <li> compre the resulting cunked data's shape, size and tasks...</li>
            </ul>
</div>

As you can see in the [graphic](../figures/chunkingprocess.png)
When selecting you may have the feeling that the chunk sizes changes. In fact Xarray will still have to fetch the entire initial dataset (LTZ) to perform the selection on any of the Data variables. Again it will not be very optimal (your Python Jupyter kernel may crash too!) with large numbers of files and large files.
This brings to our next subject [Zarr](https://zarr.readthedocs.io/en/stable/)
If we can have our original data already 'chunked' we do not need to load entire dataset.
Let's convert our test data into zarr format so that we can learn what it is.


In [ ]:
test.to_dataset().to_zarr('test.zarr',mode='w')

<div class="alert alert-warning">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Attention</b>
    <br>
    <ul>
        <li>DataArray can not be saved as 'zarr'. Before saving your data to zarr, you will need to convert that to a DataSet </li>
    </ul>
</div>

<div class="alert alert-info">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Attention</b>
    <br>
    <ul>
        <li>You can try to explore the zarr file you just created using `ls -la test.zarr` and  `ls -la test.zarr/nobs `</li>
        <li>You can explore zarr metadata file by `cat test.zarr/.zmetadata` </li>
        <li>Did you find the __chunks__ you just created in your zarr file? </li>
    </ul>
</div>

In [ ]:
!ls -al test.zarr/

In [ ]:
!cat test.zarr/.zmetadata

## opening multiple file and Kerchunk

As shown in the [Data discovery](./data_discovery.ipynb) chapter, when we have several files to read at once, we need to use Xarray `open_mfdataset`. When using open_mfdataset for example to NetCDF files, each NetCDF file is automatically considerd as 'one chunk'.  Open_mfdataset analyse each NetCDF files performs multiple operations, like concartenate the coordinate, checking compatibility,..  which is sometime time consuming, and sometime we use exactry same set of input files for different analysis.  [kerchunk library](https://fsspec.github.io/kerchunk/) can build virtual zarr libraries and can load multiple files as a set of 'zarr' file.

But that is not the only optimisation  kerchunk brings to pangeo ecosystem.



## Exploiting native chunking for reading datasets

Many data formats (for instance,  [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) or [netCDF4](https://unidata.github.io/netcdf4-python/) with HDF5 backend,[geoTIFF](https://en.wikipedia.org/wiki/GeoTIFF)) have chunk capability.  It is done at the creation of each file.  Let's call that as 'native __chunks__'.  These native chunks can be retrieved and used when opening and accessing the files. This will allow significantly reducing the amount of memory when analyzing Data Variables (only the needed chunks will be transferred, and if all the data have to be accessed it can be serializable e.g. chunks are processed one after the other).

 [kerchunk library](https://fsspec.github.io/kerchunk/) can extract these chunk information, and kerchunk can combine group of these information and can create virtual zarr libraries.

### Extract chunk information

We extract native chunk information from each NetCDF file using `kerchunk.hdf`.
Let's start with a single file.



In [ ]:
import kerchunk.hdf

We use `kerchunk.hdf` because our files are written in `netCDF4`  format which is based on HDF5 and `SingleHdf5ToZarr` to translate the metadata of one HDF5 file into Zarr metadata format. The parameter `inline_threshold` is an *optimization* and tells `SingleHdf5ToZarr` to include chunks smaller than this value directly in the output.

In [ ]:
remote_filename = 'https://object-store.cloud.muni.cz/swift/v1/foss4g-data/CGLS_LTS_1999_2019/c_gls_NDVI-LTS_1999-2019-1221_GLOBE_VGT-PROBAV_V3.0.1.nc'
with fsspec.open(remote_filename) as inf:
    h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, remote_filename, inline_threshold=100)
    chunk_info = h5chunks.translate()

Let's have a look at `chunk_info`. It is a Python dictionary so we can use `pprint` to print it nicely.

Content is a bit complicated, but it's only metadata in Zarr format indicating what's in the original file, and where the chunks of the file are located (bytes offset).

In [ ]:
from pprint import pprint
pprint(chunk_info)

<div class="alert alert-info">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Attention</b>
    <br>
    <ul>
        <li>Did you recognise the similarities with test.zarr's zarr metadata file? </li>
    </ul>
</div>

After we have collected information on the native chunks in the original data file and consolidated our Zarr metadata, we can open the files using `zarr` and pass this chunk information into a storage option. We also need to pass `"consolidated": False` because the original dataset does not contain any `zarr` consolidating metadata.

In [ ]:
LTS = xr.open_mfdataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo": chunk_info,
        },
        "consolidated": False
    }
)
LTS

As you can notice above, all the Data Variables are already chunked according to the native chunking of the NetCDF file.

### Combine all LTS files into one kerchunked single ensemble dataset

Now we will combine all the files into one kerchunked consolidated dataset, and try to open it as a xarray dataset.

Let us first collect the chunk information for each file.

In [ ]:
fs.ls('foss4g-data/CGLS_LTS_1999_2019/')

We have 36 files to process, but for this chunking_introduction example, we'll just use 6 file so that it take less time.

In [ ]:
from datetime import datetime

In [ ]:
%%time
s3path = 's3://foss4g-data/CGLS_LTS_1999_2019/c_gls_*.nc'
s3path = 's3://foss4g-data/CGLS_LTS_1999_2019/c_gls_NDVI-LTS_1999-2019-0[7-8]*.nc'
chunk_info_list = []
time_list = []

for file in fs.glob(s3path):
    url = 'https://object-store.cloud.muni.cz/swift/v1/' + file
    t = datetime.strptime(file.split('/')[-1].split('_')[3].replace('1999-', ''), "%Y-%m%d")
    time_list.append(t)
    print('working on ', file)
    with fsspec.open(url) as inf:
        h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, url, inline_threshold=100)
        chunk_info_list.append(h5chunks.translate())

This time we use `MultiZarrToZarr` to combine multiple kerchunked datasets into a single logical aggregated dataset. Like when opening multiple files with Xarray `open_mfdataset`, we need to tell `MultiZarrToZarr` how to concatenate all the files. There is no time dimension in the original dataset, but one file corresponds to one date (average over the period 1999-2019 for a given 10-day period e.g. January 01, January 11, January 21, etc.).

In [ ]:
%%time
from kerchunk.combine import MultiZarrToZarr
mzz = MultiZarrToZarr(
    chunk_info_list,
    coo_map={'INDEX': 'INDEX'},
    identical_dims=['crs'],
    concat_dims=["INDEX"],
)

out = mzz.translate()

Then, we can open the complete dataset using our consolidated Zarr metadata.

In [ ]:
%%time
LTS = xr.open_mfdataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo": out,
        },
        "consolidated": False
    }
)
LTS

We can save the consolidated metadata for our dataset in a file, and reuse it later to access the dataset.

In [ ]:
import json

In [ ]:
jsonfile='test.json'
with open(jsonfile, mode='w') as f :
    json.dump(out, f)

We can then load data from this catalog.

In [ ]:
import xarray as xr
LTS = xr.open_mfdataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo":'./test.json',
        },
        "consolidated": False
    }
)
LTS

The catalog (json file we created) can be shared on the cloud (or GitHub, etc.) and anyone can load it from there too.
This approach allows anyone to easily access LTS data and select the Area of Interest for their own study.

We have prepared json file based on 36 netcdf file, and published it online as catalogue="https://object-store.cloud.muni.cz/swift/v1/foss4g-catalogue/c_gls_NDVI-LTS_1999-2019.json"
We can try to load it.


In [ ]:
catalogue="https://object-store.cloud.muni.cz/swift/v1/foss4g-catalogue/c_gls_NDVI-LTS_1999-2019.json"
LTS = xr.open_mfdataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo":catalogue
                    },
        "consolidated": False
    }
)
LTS

We will use this catalogue in [dask_introduction](./dask_introduction.ipynb) chapter. 

## Conclusion

Understanding chunking is key to optimize your data analysis. In this episode we learned how to optimize the memory by exploiting native chunks from netCDF4 data files and instructing Xarray to access data per chunk. However, computations can be very slow and to optimize the computational part we need to parallelize our data analysis. This is what you will learn in the next episode with Dask.

<div class="alert alert-success">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Key Points</b>
    <br>
    <ul>
        <li>Chunking </li>
        <li>zarr </li>
        <li>kerchunk</li>
    </ul>
</div>

## Packages citation

```{bibliography}
:style: alpha
:filter: topic % "chunking" and topic % "package"
:keyprefix: d-
```